# 9_SVM_Classification.ipynb

In [1]:
# --- Imports ---
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_auc_score, precision_score, recall_score, f1_score
)

In [2]:
# --- Load dataset ---
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

In [3]:
# --- Scale ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [4]:
# --- PCA ---
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X_scaled)

In [5]:
# --- Split ---
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

In [6]:
# --- SVM Model (with probability=True for AUC) ---
svm = SVC(kernel='rbf', probability=True, random_state=42)
svm.fit(X_train, y_train)

SVC(probability=True, random_state=42)

In [7]:
y_pred = svm.predict(X_test)
y_prob = svm.predict_proba(X_test)[:, 1]

In [8]:
# --- Evaluation ---
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
cm = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC: {auc:.4f}")

Confusion Matrix:
 [[41  2]
 [ 2 69]]

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.95      0.95        43
           1       0.97      0.97      0.97        71

    accuracy                           0.96       114
   macro avg       0.96      0.96      0.96       114
weighted avg       0.96      0.96      0.96       114

Accuracy: 0.9649
Precision: 0.9718
Recall: 0.9718
F1 Score: 0.9718
AUC: 0.9964


In [9]:
# --- k-Fold CV ---
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(svm, X_pca, y, cv=kfold, scoring='accuracy')

print("\nCross-validation Accuracy Scores:", cv_scores)
print("Average CV Accuracy:", np.mean(cv_scores))


Cross-validation Accuracy Scores: [0.96491228 0.93859649 0.96491228 0.96491228 0.98230088]
Average CV Accuracy: 0.9631268436578171
